In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [12]:
import numpy as np
import tensorflow as tf
import datetime

from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.datasets import mnist
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
import tensorflow as tf
import torch.nn.functional as F
import torchvision 
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import copy
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Clear any logs from previous runs
%rm -rf ./logs/

UsageError: Line magic function `%rm` not found.


In [4]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

Get the dataset and insert it into relevent variables

In [13]:
#Download the dataset
def download():

    transform = transforms.ToTensor()

    batch_size = 64

    train_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)


    train_dataset_mnist = torch.utils.data.Subset(train_dataset_mnist,list(range(800)))

    val_dataset_mnist = torch.utils.data.Subset(train_dataset_mnist,list(range(800, 1000)))


    #train_dataset_mnist,val_dataset_mnist = random_split(train_dataset_mnist, [55000, 5000])
    train_loader_mnist = torch.utils.data.DataLoader(train_dataset_mnist, batch_size=batch_size, shuffle=True, num_workers=2)

    val_loader_mnist = torch.utils.data.DataLoader(val_dataset_mnist, batch_size=batch_size, shuffle=True, num_workers=2)
                                            
    test_loader_mnist = torch.utils.data.DataLoader(test_dataset_mnist, batch_size=batch_size, shuffle=False, num_workers=2)

    X_train, y_train = next(iter(train_loader_mnist))
    X_test, y_test = next(iter(test_loader_mnist))

    print(X_train[2], y_train.shape)

    print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
    print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))

    #one hot encode target values
    trainy = to_categorical(y_train)
    testy = to_categorical(y_test)
    
    return X_train, trainy, X_test, testy


In [6]:
#preprocess the dataset in it's relevent way
def prep_pixels(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')

    #normalize
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    return train_norm, test_norm

In [7]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation=LeakyReLU(), input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation=LeakyReLU()))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation=LeakyReLU()))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = Adam(lr=0.0001)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [14]:
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = download()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()

	log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
	tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

	# fit model
	model.fit(trainX, trainY, epochs=3, batch_size=64, validation_data=(testX, testY), verbose=0, callbacks=[tensorboard_callback])
	# evaluate model
	_, acc = model.evaluate(testX, testY)
	print('> %.3f' % (acc * 100.0))
	# learning curves

In [15]:
run_test_harness()

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

AttributeError: 'Tensor' object has no attribute 'astype'